# Imports

In [1]:
from typing import List

import re
import string

import numpy as np
import pandas as pd
pd.set_option('max_colwidth', 150)

from stop_words import get_stop_words
from nltk import ngrams
from bulstem.stem import BulStemmer 
from lemmagen3 import Lemmatizer 

In [2]:
DATA_PATH_PREP = '../DATA/prepared'

STEM_RULES = './stem_rules_context_2_utf8.txt'

stop_words = get_stop_words('bulgarian')
lemmatizer = Lemmatizer('bg')
stemmer = BulStemmer.from_file(STEM_RULES, min_freq=2, left_context=2)

# Load data

In [3]:
df_full = pd.read_pickle(f'{DATA_PATH_PREP}/02_df_full_text_no_meta.pkl')
df_full.head()

,author,title,download_link,local_filename,text
0,ivan_vazov,Чичовци,https://chitanka.info/text/3757-chichovtsi.txt.zip,../DATA/original/ivan_vazov_tbc_chichovtsi.txt,\n\tГалерия от типове и нрави български в турско време\n\n\n\tI. Общество\n\n\tУтринното лятно слънце изскокна високо над Стара планина. Потоци от...
1,ivan_vazov,Под игото,https://chitanka.info/text/3753-pod-igoto.txt.zip,../DATA/original/ivan_vazov_tbc_pod_igoto.txt,"\n\n\n\tЧаст първа\n\n\tI. Гост\n\n\tТая прохладна майска вечер чорбаджи Марко, гологлав, по халат, вечеряше с челядта си на двора.\n\tГосподарска..."
2,ivan_vazov,Българският език,https://chitanka.info/text/5189-bylgarskijat-ezik.txt.zip,../DATA/original/ivan_vazov_bylgarskijat_ezik.txt,"\n\n\n\n\tЕзик свещен на моите деди\n\tезик на мъки, стонове вековни,\n\tезик на тая, дето ни роди\n\tза радост не — за ядове отровни.\n\n\tЕзик п..."
3,ivan_vazov,Кочо,https://chitanka.info/text/3851-kocho.txt.zip,../DATA/original/ivan_vazov_kocho.txt,"\n\tЗащитата на Перущица\n\n\n\tO, движенье славно, о, мрачно движенье,\n\tдни на борба горда, о, дни на паденье!\n\tЕпопея тъмна, непозната нам,\..."
4,ivan_vazov,Левски,https://chitanka.info/text/3849-levski.txt.zip,../DATA/original/ivan_vazov_levski.txt,"\n\n\n\tМанастирът тесен за мойта душа е.\n\tКога човек дойде тук да се покае,\n\tтрябва да забрави греховния мир,\n\tда бяга съблазни и да търси ..."


# Create samples

In [4]:
df_group_by_author = df_full.groupby("author")["text"].agg(" ".join).reset_index()
df_group_by_author

,author,text
0,aleko-konstantinov,"\n\n\n\tНашият помощник на регистратора е страшен комик. Изпокапваме от смях, когато почне да бомбардира несправедливата съдба.\n\t— Е, дявол да г..."
1,dimityr-dimov,"\n\n\n\tI глава\n\n\tГроздоберът беше към края си.\n\tОт малките вили и кирпичените постройки сред лозята се разнасяха ту задружни песни, бързи и ..."
2,dimityr-talev,"\n\n\n\tПърва част\n\tХаджи Серафимовата внука\n\n\n\n\tОвде дърво столовито,\n\tстоловито, грановито,\n\tгранки му са до небеси,\n\tа корени — су..."
3,elin-pelin,"\n\n\n\tПърва глава\n\n\tНай-заможният човек в селото беше дядо Йордан Геракът. Пъргав и трудолюбив, той бе работил през целия си живот и бе сполу..."
4,ivan_vazov,\n\tГалерия от типове и нрави български в турско време\n\n\n\tI. Общество\n\n\tУтринното лятно слънце изскокна високо над Стара планина. Потоци от...
5,jordan-jovkov,"\n\n\n\tI\n\n\tГрадът бързо се подновяваше и растеше. Но все пак оставаха много улици, където още можеше да се видят тихи, едновремешни къщурки, с..."


In [5]:
limit_x = df_group_by_author['text'].map(lambda t: len(t.strip().split())).min() - 1000
print(f'{limit_x=}')
limit_y = 150_000

num_authors = df_group_by_author.shape[0]
num_samples = 1000

start_idxs = np.random.randint(0, limit_x, (num_authors, num_samples))
sample_lengths = np.random.randint(0, limit_y, (num_authors, num_samples))

df_samples = pd.DataFrame(columns=['text', 'author'])
for i in range(num_authors):
    author_name = df_group_by_author['author'][i]
    for j, start_idx in enumerate(start_idxs[i]):
        sample = df_group_by_author['text'][i][start_idx : (start_idx + sample_lengths[i, j])]
        df_samples.loc[len(df_samples)] = [sample, author_name]
df_samples

limit_x=21623


,text,author
0,"е курдисам аз тебе на митницата… Тука ли си, сама ли си!… Че да ги пипна аз ония ми ти търговци, две годинки да им обирам каймака — стига ми! Па с...",aleko-konstantinov
1,"ки, та виж няма ли да те приемат. Защо се обаждаш, защо не си мълчиш; теб какво ти влиза в работата, че този бил подлец, па оня бил крадец, па Бял...",aleko-konstantinov
2,"то милеят толкова за нея. Еле тази солунска митница — на сърцето ми е израснала, ваджията! Ех, че келепир, майка му мечка!… Да ще султанът да ни д...",aleko-konstantinov
3,"itanka.info/text/3709\n\tОригинален източник: [[http://slovo.bg|Словото]]\n\tДопълнителна корекция: zelenkroki, 2017\n\t----\n\t__Издание:__\n\tВе...",aleko-konstantinov
4,"ереш бе, брате; ами че, моля ви се, вземете например кокона Поликсени: преди трийсет години с фаетон се разхождаше и днес, моля ви се, пак с фаето...",aleko-konstantinov
...,...,...
5995,"или пък самата му орисия беше такава, но той беше съвсем негоден за каквато и да било работа. Някакъв откършлек от безгрижната и весела бохема на...",jordan-jovkov
5996,яскаво и галантно са флиртували из широките зали на замъците. Готовите рецепти на тая наука за сърцето инженерът беше намерил или в някой отдавна ...,jordan-jovkov
5997,"и в околността. По една отколешна традиция, която заможните семейства още пазеха, Цветана се учеше в Роберт колеж в Цариград, но всяка ваканция пр...",jordan-jovkov
5998,"и делничен ден. В града кипеше трескава работа. Улиците бяха задръстени от кола, всички магазини бяха отворени, около изложената навън стока се тр...",jordan-jovkov


# Tokenize

In [6]:
def tokenize(raw_text: str, level: str) -> List[str]:
    if level not in {'soft', 'medium', 'hard'}:
        raise NotImplemented(f'Level {level} is not supported')
    
    text = raw_text.lower()
    
    if level in {'hard'}:
        text = text.translate(text.maketrans('', '', string.punctuation))  # Remove punctuation.
        text = re.sub(r'[a-zA-Z]', "", text)  # Remove non-bulgarian words.
        
        tokens = text.split()  # Split on whitespace
        tokens = [token for token in tokens if token not in stop_words  # Filter out stopwords
              and all(c.isalpha() for c in token)]  # and non-word tokens.
    else:
        # Split on punctuation and digits and keep them.
        tokens = re.findall(r"[\w']+|[^\w\s]", text)

    if level in {'medium', 'hard'}:
        # Now: ['песни', 'македония', 'българският', 'бог', ..
        tokens = [lemmatizer.lemmatize(token) for token in tokens]

    if level in {'hard'}:
        # Now: ['песен', 'македония', 'български', 'бог', ..
        tokens = [stemmer.stem(token) for token in tokens]
        # Now: ['песен', 'македони', 'българск', 'бог', ..

    bi_tri_grams = list(ngrams(tokens, 2)) + list(ngrams(tokens, 3)) + list(ngrams(tokens, 4))
    tokens += map(lambda bts: ' '.join(bts), bi_tri_grams)

    return tokens

In [7]:
test = 'Това е низ! Съдържа препинателни "знаци" и цифри като: 42, 420, 4200 и 7.\nКак можем да го разделим??'

print('################ Soft ################')
print(tokenize(test, 'soft'))
print()

print('################ Medium ################')
print(tokenize(test, 'medium'))
print()

print('################ Hard ################')
print(tokenize(test, 'hard'))

################ Soft ################
['това', 'е', 'низ', '!', 'съдържа', 'препинателни', '"', 'знаци', '"', 'и', 'цифри', 'като', ':', '42', ',', '420', ',', '4200', 'и', '7', '.', 'как', 'можем', 'да', 'го', 'разделим', '?', '?', 'това е', 'е низ', 'низ !', '! съдържа', 'съдържа препинателни', 'препинателни "', '" знаци', 'знаци "', '" и', 'и цифри', 'цифри като', 'като :', ': 42', '42 ,', ', 420', '420 ,', ', 4200', '4200 и', 'и 7', '7 .', '. как', 'как можем', 'можем да', 'да го', 'го разделим', 'разделим ?', '? ?', 'това е низ', 'е низ !', 'низ ! съдържа', '! съдържа препинателни', 'съдържа препинателни "', 'препинателни " знаци', '" знаци "', 'знаци " и', '" и цифри', 'и цифри като', 'цифри като :', 'като : 42', ': 42 ,', '42 , 420', ', 420 ,', '420 , 4200', ', 4200 и', '4200 и 7', 'и 7 .', '7 . как', '. как можем', 'как можем да', 'можем да го', 'да го разделим', 'го разделим ?', 'разделим ? ?', 'това е низ !', 'е низ ! съдържа', 'низ ! съдържа препинателни', '! съдържа препин

In [8]:
df_full_soft = df_samples.copy()
df_full_soft['text'] = df_full_soft['text'].map(lambda t: tokenize(t, 'soft'))

df_full_med = df_samples.copy()
df_full_med['text'] = df_full_med['text'].map(lambda t: tokenize(t, 'medium'))

df_full_hard = df_samples.copy()
df_full_hard['text'] = df_full_hard['text'].map(lambda t: tokenize(t, 'hard'))

In [ ]:
print('################ Heads ################')
print(df_full_soft['text'][0][:20])
print(df_full_med['text'][0][:20])
print(df_full_hard['text'][0][:20])

print('################ Tails ################')
print(df_full_soft['text'][0][-20:])
print(df_full_med['text'][0][-20:])
print(df_full_hard['text'][0][-20:])

In [ ]:
df_full_hard.head()

# Saving to files

In [ ]:
df_full_soft.to_pickle(f'{DATA_PATH_PREP}/03_df_full_soft.pkl')
df_full_med.to_pickle(f'{DATA_PATH_PREP}/03_df_full_med.pkl')
df_full_hard.to_pickle(f'{DATA_PATH_PREP}/03_df_full_hard.pkl')